In [7]:
# python $SUMO_HOME/tools/randomTrips.py -n config/osm.net.xml.gz -r routes.rou.xml -e 10000 -l --insertion-density=12

In [1]:
import os
import sys
import random
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import sumolib
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import sys
import io
from contextlib import redirect_stdout


if 'SUMO_HOME' in os.environ:
    print('SUMO_HOME found')
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import traci
from sumolib import checkBinary
# sumoBinary = checkBinary('sumo-gui')
sumoBinary = checkBinary('sumo')
roadNetwork = "./config/osm.sumocfg"
sumoCmd = [sumoBinary, "-c", roadNetwork, "--start", "--quit-on-end"]
# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

SUMO_HOME found
Using device: cuda


In [2]:
def randomTrips(dur=10000, density=12):
    os.system("python $SUMO_HOME/tools/randomTrips.py -n config/osm.net.xml.gz -r config/osm.passenger.trips.xml -e " + str(dur) + " -l --insertion-density=" + str(density))

In [3]:
def shouldContinueSim():
    numVehicles = traci.simulation.getMinExpectedNumber()
    return True if numVehicles > 0 else False

In [4]:
def restart(dur, density):
    with io.StringIO() as buf, redirect_stdout(buf):
        try:
            traci.close()
        except:
            pass
        randomTrips(dur, density)
        traci.start(sumoCmd)
        step = 0
        for i in range(100):
            traci.simulationStep()
            step += 1

In [5]:
def zeros2end(x, length=None):
    mask = x == 0
    out = torch.stack(
        [torch.cat([x[_, :][~mask[_, :]], x[_, :][mask[_, :]]], dim=0) for _ in range(x.size()[0])]).to(device)
    if length is not None and length < out.size()[1]:
        out = out[:, :length]
    elif length is not None and length > out.size()[1]:
        out = torch.cat([out, torch.zeros(out.size()[0], length - out.size()[1]).to(device)], dim=1)
    return out.to(device)

In [6]:
def intervehicleConnectivity(threshold = None):
    xs = []
    ys = []
    for vehicle in traci.vehicle.getIDList():
        x, y = traci.vehicle.getPosition(vehicle)
        xs.append(x)
        ys.append(y)
    xs = torch.tensor(xs, dtype=torch.float32).to(device).view(-1,1)
    ys = torch.tensor(ys, dtype=torch.float32).to(device).view(-1,1)
    intervehicle_distances = torch.sqrt((xs - xs.t())**2 + (ys - ys.t())**2)
    if threshold is not None:
        # # make only the distances less than the threshold non-zero, the rest
        # intervehicle_distances = torch.where(intervehicle_distances < threshold, intervehicle_distances, torch.zeros_like(intervehicle_distances))
        # make the distances 1 if less than the threshold, 0 otherwise
        intervehicle_distances = torch.where(intervehicle_distances < threshold, torch.ones_like(intervehicle_distances), torch.zeros_like(intervehicle_distances))
    return intervehicle_distances, xs, ys

In [7]:
def getVehicleData(connectivity, xs, ys, length=500):
    masked_xs = torch.matmul(connectivity, torch.diag(xs.view(-1)))
    masked_ys = torch.matmul(connectivity, torch.diag(ys.view(-1)))
    vehicle_accelerations = torch.tensor([traci.vehicle.getAcceleration(vehicle) for vehicle in traci.vehicle.getIDList()], dtype=torch.float32).to(device).view(-1,1)
    vehicle_speeds = torch.tensor([traci.vehicle.getSpeed(vehicle) for vehicle in traci.vehicle.getIDList()], dtype=torch.float32).to(device).view(-1,1)
    vehicle_directions = torch.tensor([traci.vehicle.getAngle(vehicle) for vehicle in traci.vehicle.getIDList()], dtype=torch.float32).to(device).view(-1,1)
    masked_accelerations = torch.matmul(connectivity, torch.diag(vehicle_accelerations.view(-1)))
    masked_speeds = torch.matmul(connectivity, torch.diag(vehicle_speeds.view(-1)))
    masked_directions = torch.matmul(connectivity, torch.diag(vehicle_directions.view(-1)))
    vehicle_data = torch.cat([masked_xs.unsqueeze(2), masked_ys.unsqueeze(2), masked_accelerations.unsqueeze(2), masked_speeds.unsqueeze(2), masked_directions.unsqueeze(2)], dim=2)
    vehicle_data = vehicle_data.view(vehicle_data.size(0), -1)
    vehicle_data = zeros2end(vehicle_data, length)
    return vehicle_data

In [8]:
def getLabels(prev_connectiviy, prev_ids, length=500):
    # get all vehicle's current x, y positions
    curr_ids = traci.vehicle.getIDList()
    removed_ids = list(set(prev_ids) - set(curr_ids))
    xs = []
    ys = []
    for vehicle in prev_ids:
        if vehicle in removed_ids:
            xs.append(-1)
            ys.append(-1)
        else:
            x, y = traci.vehicle.getPosition(vehicle)
            xs.append(x)
            ys.append(y)
    xs = torch.tensor(xs, dtype=torch.float32).to(device).view(-1,1)
    ys = torch.tensor(ys, dtype=torch.float32).to(device).view(-1,1)
    # do pointwise multiplication of xs with prev_connectivity
    masked_xs = torch.mul(prev_connectiviy, xs).T
    masked_ys = torch.mul(prev_connectiviy, ys).T
    vehicle_labels = torch.cat([masked_xs.unsqueeze(2), masked_ys.unsqueeze(2)], dim=2).to(device)
    vehicle_labels = vehicle_labels.view(vehicle_labels.size(0), -1).to(device)
    vehicle_labels = zeros2end(vehicle_labels, length).to(device)
    return vehicle_labels

## Define the model (without GNN)

In [12]:
class TrafficPredictionModel(torch.nn.Module):
    def __init__(self, neigh_vec_dim, hidden, output_dim):
        super(TrafficPredictionModel, self).__init__()
        self.fc1 = torch.nn.Linear(neigh_vec_dim, hidden)
        self.fc2 = torch.nn.Linear(hidden, output_dim)

    def forward(self, neigh_vehicles_vector):
        x = F.relu(self.fc1(neigh_vehicles_vector))
        x = F.relu(self.fc2(x))
        return x

## Train the NON-GNN network

In [24]:
# Generate synthetic data
def generate_data(vehicle_data_length, interval_steps=5, dsrc_range=500, dur=5000, density=6):
    restart(dur, density)
    step = 0
    inputs = []
    targets = []
    while step < dur and shouldContinueSim():
        connectivity, xs, ys = intervehicleConnectivity(dsrc_range)
        neigh_vehicles_vector = getVehicleData(connectivity, xs, ys, vehicle_data_length * 5).to(device)
        old_ids = traci.vehicle.getIDList()

        for i in range(interval_steps):
            traci.simulationStep()
            step += 1

        target_positions = getLabels(connectivity, old_ids, vehicle_data_length * 2).to(device)
        inputs.append(neigh_vehicles_vector[0])
        targets.append(target_positions[0])
    inputs = torch.stack(inputs)
    targets = torch.stack(targets)
    dataset = TensorDataset(inputs, targets)  # Create a TensorDataset
    return DataLoader(dataset, batch_size=100, shuffle=True)
        

# Training the model
def train(model, criterion, optimizer, num_epochs, vehicle_data_length, interval_steps=5, dsrc_range=500, dur=5000, density=6):
    model.train()
    for epoch in range(num_epochs):
        # Get new data each epoch
        dataloader = generate_data(vehicle_data_length, interval_steps, dsrc_range, dur, density)
        total_loss = 0
        for inputs, targets in dataloader:
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs, targets)

            # Backward pass
            loss.backward()

            # Update parameters
            optimizer.step()

            total_loss += loss.item()

        if (epoch+1) % 10 == 0:
            avg_loss = total_loss / len(dataloader)
            print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')


In [25]:
# Instantiate the model
dur = 5000
density = 6
vehicle_data_length = 20
dsrc_range = 5000
interval_steps = 5
restart(dur, density)

connectivity, xs, ys = intervehicleConnectivity(dsrc_range)
neigh_vehicles_vector = getVehicleData(connectivity, xs, ys, vehicle_data_length * 5).to(device)
old_ids = traci.vehicle.getIDList()
step = 0
for i in range(interval_steps):
    traci.simulationStep()
    step += 1

target_positions = getLabels(connectivity, old_ids, vehicle_data_length * 2).to(device)
model = TrafficPredictionModel(neigh_vec_dim=neigh_vehicles_vector.shape[1], hidden=128, output_dim=target_positions.shape[1]).to(device)
# Define loss and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
train(model, criterion, optimizer, num_epochs, vehicle_data_length, interval_steps, dsrc_range, dur, density)

Simulation ended at time: 5100.00
Reason: TraCI requested termination.
Performance: 
 Duration: 1308.39s
 TraCI-Duration: 3.99s
 Real time factor: 3.89793
 UPS: 59.103308
Vehicles: 
 Inserted: 380
 Running: 9
 Waiting: 0
Statistics (avg of 371):
 RouteLength: 1584.57
 Speed: 7.75
 Duration: 204.35
 WaitingTime: 11.12
 TimeLoss: 36.81
 DepartDelay: 0.53

Success.
***Starting server on port 45023 ***
Loading net-file from './config/osm.net.xml.gz' ... done (67ms).
Loading additional-files from './config/osm.poly.xml.gz' ... done (161ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.
Simulation ended at time: 105.00
Reason: TraCI requested termination.
Performance: 
 Duration: 0.40s
 TraCI-Duration: 0.02s
 Real time factor: 265.152
 UPS: 1401.515152
Vehicles: 
 Inserted: 10 (Loaded: 18)
 Running: 10
 Waiting: 0
Statistics (avg of 0):
 RouteLength: 0.00
 Speed: 0.00
 Duration: 0.00
 WaitingTime: 0.00
 TimeLoss: 0.00
 DepartDelay: 0.00

Success.
***Starting server on port

KeyboardInterrupt: 

In [ ]:
def check(i):
    model.eval()
    with torch.no_grad():
        prediction = model(neigh_vehicles_vector[i])
        print(prediction)
        print(target_positions[i])
check(15)

tensor([ 8.1872e+02,  4.9300e+02,  6.7327e+02,  4.0129e+02,  1.0266e+03,
         6.1192e+02,  1.0235e+03,  5.7670e+02,  1.2238e+03,  5.9713e+02,
         1.0596e+03,  5.5669e+02,  9.8745e+02,  5.1876e+02,  8.3305e+02,
         4.0075e+02,  8.1763e+02,  3.7739e+02,  1.0219e+03,  4.5800e+02,
         8.1804e+02,  3.6240e+02,  8.8421e+02,  4.0961e+02,  7.6787e+02,
         3.7227e+02,  7.9919e+02,  3.0671e+02,  1.0303e+03,  4.2392e+02,
         9.4643e+02,  3.8858e+02,  8.1903e+02,  4.5717e+02,  8.7270e+02,
         3.9464e+02,  9.0492e+02,  3.7391e+02,  8.3559e+02,  4.0438e+02,
         9.1100e+02,  3.9342e+02,  9.4737e+02,  4.2318e+02,  8.1770e+02,
         3.8515e+02,  7.9423e+02,  4.0380e+02,  8.4357e+02,  3.9070e+02,
         8.4944e+02,  3.9370e+02,  8.7002e+02,  4.7568e+02,  8.0487e+02,
         4.4310e+02,  8.8095e+02,  4.0253e+02,  7.6027e+02,  4.2968e+02,
         8.9714e+02,  3.7803e+02,  8.3495e+02,  3.8067e+02,  9.1753e+02,
         4.5364e+02,  8.0439e+02,  3.5815e+02,  9.1

In [73]:
class TrafficPredictionModel(torch.nn.Module):
    def __init__(self, neigh_vec_dim, hidden, output_dim):
        super(TrafficPredictionModel, self).__init__()
        self.fc1 = torch.nn.Linear(neigh_vec_dim, hidden)
        self.fc2 = torch.nn.Linear(hidden, hidden)
        self.fc3 = torch.nn.Linear(hidden, hidden)
        self.fc4 = torch.nn.Linear(hidden, output_dim)

    def forward(self, neigh_vehicles_vector):
        x = F.relu(self.fc1(neigh_vehicles_vector))
        # x = F.relu(self.fc2(x))
        # x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return x

In [23]:
class TrafficPredictionModel(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, kernel_size=2, stride=2, padding=0):
        super(TrafficPredictionModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)

    def forward(self, x):
        x = self.conv1(x)
        return F.relu(x)

In [95]:
i = 100
model = TrafficPredictionModel().to(device)

# Loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error Loss
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Generate synthetic data
def generate_data(num_samples, i=1):
    inputs = torch.rand(num_samples*i, 2) * 10
    targets = inputs.sum(dim=1, keepdim=True)  # Sums of each pair
    inputs = inputs.view(num_samples, 1, 2 * i).to(device)
    targets = targets.view(num_samples, i).to(device)
    dataset = TensorDataset(inputs, targets)  # Create a TensorDataset
    return DataLoader(dataset, batch_size=1, shuffle=True)  # Create a DataLoader for batching

# Training the model
def train(model, criterion, optimizer, num_epochs, num_samples, i):
    model.train()
    for epoch in range(num_epochs):
        dataloader = generate_data(num_samples, i)  # Get DataLoader
        total_loss = 0
        for inputs, targets in dataloader:
            # Zero the gradients
            optimizer.zero_grad()
            # Forward pass
            outputs = model(inputs)
            # Compute loss
            loss = criterion(outputs, targets)

            # Backward pass
            loss.backward()

            # Update parameters
            optimizer.step()

            total_loss += loss.item()

        if (epoch+1) % 10 == 0:
            avg_loss = total_loss / len(dataloader)
            print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')
            if avg_loss < 1e-2:
                break

# Parameters
num_epochs = 1000
num_samples = 100

# Train the model
train(model, criterion, optimizer, num_epochs, num_samples, i)

Epoch [10/1000], Average Loss: 0.0000
